Some useful links from Dana:

https://scikit-image.org/docs/stable/auto_examples/edges/plot_skeleton.html

https://nalinc.github.io/blog/2018/skin-detection-python-opencv/

https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html

The paper and video we're basing everythign off of:

https://link.springer.com/article/10.1007/s11042-013-1501-1

https://www.youtube.com/watch?v=xML2S6bvMwI

In [1]:
import numpy as np
import math
import cv2
cap = cv2.VideoCapture(1)

# skin colour numbers
y_min = 64
y_max = 193
cr_min = 139 # 133
cr_max = 173
cb_min = 77
cb_max = 127


while 1:
    _ret, img = cap.read()
    #cv2.rectangle(img, (700, 700), (100, 100), (0, 255, 0), 0)
    crop_img = img#img[100:700, 100:700]
    drawing = np.zeros(crop_img.shape, np.uint8)
    #grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    kernel_size = (35, 35)
    
    img_ycrcb = cv2.cvtColor(crop_img, cv2.COLOR_BGR2YCrCb)
    #blurred = cv2.GaussianBlur(grey, kernel_size, 0)
    blurred = cv2.GaussianBlur(img_ycrcb, kernel_size, 0)

    scale_factor = 4
    if scale_factor != 1:
        blurred = cv2.resize(blurred, [blurred.shape[1]//scale_factor, blurred.shape[0]//scale_factor], interpolation=cv2.INTER_AREA)
    
    # skin color segmentation
    skin_ycrcb_mint = np.array([y_min, cr_min, cb_min], np.uint8)
    skin_ycrcb_maxt = np.array([y_max, cr_max, cb_max], np.uint8)
    thresholded = cv2.inRange(blurred, skin_ycrcb_mint, skin_ycrcb_maxt)
    #_ret, thresholded = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#cv2.CHAIN_APPROX_NONE)

    # if no contours are detected, don't draw anything
    if len(contours) != 0:
        count1 = max(contours, key=lambda x: cv2.contourArea(x))
            
        x, y, w, h = cv2.boundingRect(count1)

        rotRect = cv2.minAreaRect(count1)
        box = cv2.boxPoints(rotRect)
        box = np.int64(box)
        cv2.drawContours(crop_img, [box*scale_factor], 0, (255, 255, 0), 1)

        cv2.rectangle(crop_img, (x*scale_factor, y*scale_factor), ((x + w)*scale_factor, (y + h)*scale_factor), (0, 0, 255), 1)
        hull = cv2.convexHull(count1)
        cv2.drawContours(drawing, [count1*scale_factor], 0, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull*scale_factor], 0, (0, 0, 255), 0)
        try:
            hull = cv2.convexHull(count1, returnPoints=False)
            defects = cv2.convexityDefects(count1, hull)
        except:
            print(hull, count1)
            continue
            raise IOError("Hull issue")
        count_defects = 0
        cv2.drawContours(thresholded, contours*scale_factor, -1, (0, 255, 0), 3)

        center_pt = (0, 0)
        center_dist = 0
        # finding the center of the palm
        for pt_x in range(x, x+w, 2):
            for pt_y in range(y, y+h, 2):
                pt = (pt_x, pt_y)
                dist = cv2.pointPolygonTest(count1, pt, measureDist=True)
                if dist > center_dist:
                    center_pt = (pt_x * scale_factor, pt_y * scale_factor)
                    center_dist = dist
                #cv2.circle(crop_img, pt, 1, (255, 0, 255), 1)
                #else:
                #    cv2.circle(crop_img, pt, 1, (0, 0, 0), 1)
        # biggest circle in palm
        cv2.circle(crop_img, center_pt, int(center_dist)*scale_factor, (255, 0, 0), 2)
        # circle 3.5 * the radius
        #cv2.circle(crop_img, center_pt, int(center_dist*3.5)*scale_factor, (255, 255, 0), 2)
        # biggest circle / 2
        cv2.circle(crop_img, center_pt, int(center_dist//2)*scale_factor, (0, 165, 255), 2)
        # center of biggest circle
        cv2.circle(crop_img, center_pt, 1, (120, 0, 0), 2)

        big_c, big_r = cv2.minEnclosingCircle(count1)
        big_c = (int(big_c[0])*scale_factor, int(big_c[1])*scale_factor)

        if ((big_r*scale_factor) > (3.5* center_dist*scale_factor)): 
            cv2.circle(crop_img, big_c, int(big_r)*scale_factor, (0, 0, 255), 2)
        

        # doing contour extractions once again within the 3.5x circle
        # grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        # blurred2 = cv2.GaussianBlur(grey, kernel_size, 0)
        # _ret, thresholded2 = cv2.threshold(blurred2, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # contours2, _hierarchy = cv2.findContours(thresholded2.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#cv2.CHAIN_APPROX_NONE)

        # if len(contours2) != 0:
        #     count1_2 = max(contours2, key=lambda x: cv2.contourArea(x))


        # print(len(contours), defects.shape, '\n\n', contours)
        # break

        if defects is not None:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(count1[s][0])
                end = tuple(count1[e][0])
                far = tuple(count1[f][0])
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 57

                if angle <= 90:
                    count_defects += 1
                    cv2.circle(crop_img, (far[0]*scale_factor, far[1]*scale_factor), 3, [0, 0, 255], -1)

                cv2.line(crop_img, (start[0]*scale_factor, start[1]*scale_factor), (end[0]*scale_factor, end[1]*scale_factor), [0, 255, 0], 2)

        #cv2.putText(crop_img, f"convex: {cv2.isContourConvex(count1)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

        if count_defects < 5:
            cv2.putText(img, f"{count_defects+1} finger(s)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0))

    #cv2.imshow('main window', img)
    all_img = np.hstack((drawing, crop_img))
    cv2.imshow('Contours', all_img)

    k = cv2.waitKey(1)
    if k == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

2024-03-30 11:38:37.535 Python[2347:56021] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-03-30 11:38:39.048 Python[2347:56021] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
